In [13]:
import re, pickle, os, string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np 
import pandas as pd 
import nltk
import string
import spacy
from nltk.corpus import stopwords
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.tokenizer import Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import csv

In [2]:
train_data = None
test_data = None

def load_data():
    global train_data, test_data
    train_data = pd.read_csv('train_tweets.txt', delimiter="\t", header = None, quoting=csv.QUOTE_NONE)
    test_data = pd.read_csv('test_tweets_unlabeled.txt', delimiter="\t", header = None, quoting=csv.QUOTE_NONE)
    
load_data()
train_data.columns = ['label','Tweet']
test_data.columns = ['Tweet']

print(train_data)
print(test_data)

        label                                              Tweet
0        8746     @handle Let's try and catch up live next week!
1        8746  Going to watch Grey's on the big screen - Thur...
2        8746  @handle My pleasure Patrick....hope you are well!
3        8746  @handle Hi there! Been traveling a lot and lot...
4        8746  RT @handle Looking to Drink Clean & Go Green? ...
5        8746  RT @handle: Ft. Hood officials confirm the 2 o...
6        8746  RT @handle: Mickey Mouse is Getting a Make Ove...
7        8746           @handle How did u get the invite Justin?
8        8746  @handle I think I am still a good friend of he...
9        8746  @handle I remember! I am fine - how are u? Wha...
10       8746     @handle That's great - good for the coach!!!!!
11       8746  @handle I don't want to picture u sitting on i...
12       8746  @handle D- Thanks for the RTs....are you going...
13       8746           @handle Grrr....you must be going crazy!
14       8746  @handle Hi

In [3]:
### computing the number of label
numLabel = train_data['label'].unique()
print(len(numLabel))

9297


In [4]:
# ### bag-of-words feature matrix
# from sklearn.feature_extraction.text import CountVectorizer
       
# bow_vectorizer = CountVectorizer(max_features = 30)

# allData = train_data['Tweet'].values.tolist() + test_data['Tweet'].values.tolist()
# #print("allData="+str(allData))

# bowAllData = bow_vectorizer.fit_transform(allData) 
# #bowTrain = bow_vectorizer.fit_transform(train_data['Tweet']) 
# #bowTest = bow_vectorizer.fit_transform(test_data['Tweet']) 

# #bowArray = bowTrain.toarray()
# #get_feature_names()可获取词袋中所有文本的关键字
# #toarray()可看到词频矩阵的结果
        
# train_tagged = bow_vectorizer.get_feature_names()
# print (bow_vectorizer.get_feature_names())


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 74195)
allData = train_data['Tweet'].values.tolist() + test_data['Tweet'].values.tolist()
bowAllData = vectorizer.fit_transform(allData) 
train_tagged = vectorizer.get_feature_names()
# print (bow_vectorizer.get_feature_names())
# print (len(bowAllData))

In [11]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
# train_bow = bow[:328195, :]
# xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train_data['label'], test_size=0.2)
# lsvc=LinearSVC()
# lsvc.fit(xtrain_bow, ytrain)
# prediction = lsvc.predict_proba(xvalid_bow)
# predictLabel = lsvc.predict(xvalid_bow)
# print(lsvc.predict(xvalid_bow))
lentrain = len(train_data)

# Separate back into training and test sets.
train = bowAllData[:lentrain]  
test = bowAllData[lentrain:]
# print (train)
# print (test)
#trainLabel = str(train_data['label'].unique())
#print("label="+str(train_data['label'].unique()))
#print(lreg.classes_) 
# ==========================
lsvc=LinearSVC(C=0.9)
lsvc.fit(train, train_data['label']) # training the model
 
# trainPrediction = lsvc.predict_proba(test) # predicting on the validation set

prediction = lsvc.predict(test)


In [11]:
def save_pickle(data, filepath):
    save_documents = open(filepath, 'wb')
    pickle.dump(data, save_documents, protocol = 4)
    save_documents.close()

In [12]:
save_pickle(lsvc, os.path.join('tfidf7_4195WLSVC.p'))

In [14]:
def load_pickle(filepath):
    documents_f = open(filepath, 'rb')
    file = pickle.load(documents_f)
    documents_f.close()
    
    return file

In [16]:
load_pickle('tfidf6WLSVC.p')

EOFError: Ran out of input

In [12]:
test_data['Predicted'] = prediction
submission = test_data[['Predicted']]
submission.index = np.arange(1, len(submission) + 1)
submission['Id'] = submission.index

columnsTitles=["Id","Predicted"]
submission=submission.reindex(columns=columnsTitles)
submission.to_csv('ResultTFIDF7_4195WLSVN（C=0.9）.csv',index=0)
print(submission)

          Id  Predicted
1          1       6778
2          2       8119
3          3       6492
4          4       8894
5          5       8297
6          6       4273
7          7       1130
8          8       8646
9          9        476
10        10       9390
11        11        640
12        12       1624
13        13       1071
14        14       4897
15        15       5565
16        16       4426
17        17       4450
18        18       1698
19        19        391
20        20       8058
21        21       8031
22        22         25
23        23       7022
24        24       8704
25        25       8250
26        26       7951
27        27       1205
28        28       8739
29        29       5807
30        30       2491
...      ...        ...
35408  35408       1606
35409  35409       6148
35410  35410       1463
35411  35411       4397
35412  35412       7170
35413  35413       8720
35414  35414       7265
35415  35415       8151
35416  35416       4335
35417  35417    